In [7]:
import pandas as pd
from pytrends.request import TrendReq
import time
import json

# Load the JSON file
with open('lalal.json', 'r') as file:
    uptownfunk_data = json.load(file)

# Extract unique words from the JSON content
keywords = list(set(word.lower() for entry in uptownfunk_data if 'text' in entry for word in entry['text'].split()))
regions = [
    "IN" 
]
timeframe = "now 7-d"  

In [8]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
custom_stop_words = {'ago', 'am', 'i', 'you', 'me', 'your', 'it', 'for', 'to', 'of', 'in', 'on', 'with', 
                     'the', 'this', 'that', 'my', 'we', 'who', 'if', 'about', 'up', 'out', 'got', 
                     'get', 'what', 'a', 'also', 'from', 'and', 'or', 'but', 'so', 'just', 'at',
                     'be', 'is', 'are', 'was', 'were', 'been', 'do', 'does', 'did', 'will', 'would',
                     'should', 'can', 'could', 'may', 'might', 'must', 'shall', 'having', 'has', 'had',
                     'how', 'why', 'when', 'where', 'which', 'whom', 'whose', 'here', 'there', 'then',
                     'than', 'now', 'over', 'under', 'between', 'before', 'after', 'while', 'during',
                     'against', 'through', 'though', 'since', 'because', 'until', 'unless', 'nor', 'not',
                     'no', 'yes', 'yeah', 'yep', 'yup', 'ok', 'okay', 'sure', 'well', 'good', 'great',
                     'nice', 'awesome', 'cool', 'bad', 'terrible', 'horrible', 'awful', 'wonderful', 'fantastic',}
stop_words.update(custom_stop_words)

def clean_keyword(keyword):
    keyword = re.sub(r'[^\w\s]', '', keyword)
    keyword = keyword.lower()
    if keyword in stop_words or len(keyword) < 2:
        return None
    return keyword

keywords = {clean_keyword(kw) for kw in keywords if clean_keyword(kw)}

In [9]:
keywords

{'12',
 '120',
 '17',
 '19',
 '246',
 '70',
 '80',
 '811',
 '__â',
 'accept',
 'ah',
 'aisle',
 'almonds',
 'already',
 'always',
 'anniversary',
 'another',
 'applause',
 'babe',
 'baby',
 'bankai',
 'basic',
 'bean',
 'beans',
 'bears',
 'believe',
 'better',
 'bezos',
 'big',
 'bit',
 'black',
 'boss',
 'bottle',
 'bottom',
 'box',
 'brand',
 'brings',
 'broth',
 'brother',
 'business',
 'calling',
 'cannot',
 'cant',
 'carry',
 'cents',
 'check',
 'cheese',
 'cheetos',
 'chicken',
 'child',
 'class',
 'click',
 'clone',
 'code',
 'color',
 'come',
 'comparison',
 'corn',
 'course',
 'covered',
 'cupcakes',
 'daddy',
 'daily',
 'dare',
 'david',
 'derivative',
 'diet',
 'different',
 'direct',
 'disgusting',
 'doctor',
 'doesnt',
 'dollars',
 'dont',
 'dragonball',
 'earliest',
 'earth',
 'eat',
 'ecstasy',
 'egg',
 'elements',
 'essential',
 'ever',
 'every',
 'everythings',
 'excellent',
 'excuse',
 'extra',
 'farmers',
 'feed',
 'feet',
 'find',
 'flavor',
 'foods',
 'form',
 'fo

In [10]:
def chunk_keywords(keywords, chunk_size):
    """
    Splits a list of keywords into chunks of a specified size.
    
    Args:
        keywords (list): List of keywords to split.
        chunk_size (int): Size of each chunk.
    
    Returns:
        list: List of keyword chunks.
    """
    for i in range(0, len(keywords), chunk_size):
        yield keywords[i:i + chunk_size]

In [11]:
def analyze_search_trends(keywords, regions, timeframe):
    """
    Analyzes search trends for given keywords and regions with delays to avoid rate limits.
    
    Args:
        keywords (list): List of keywords to analyze (up to 5 per request).
        regions (list): List of region codes (e.g., 'US', 'UK').
        timeframe (str): Timeframe for trends (e.g., 'today 5-y').
    
    Returns:
        dict: Trend data for each region.
    """
    pytrends = TrendReq(hl='en-US', tz=360)  # Initialize pytrends
    all_data = {}

    # Split keywords into chunks of 5
    keyword_chunks = chunk_keywords(keywords, chunk_size=5)

    flag = 0

    for region in regions:
        for chunk in keyword_chunks:
            try:
                flag += 1
                # Build payload and fetch data
                pytrends.build_payload(chunk, cat=0, timeframe=timeframe, geo=region)
                data = pytrends.interest_over_time()
                if not data.empty:
                    if region not in all_data:
                        all_data[region] = data
                    else:
                        all_data[region] = all_data[region].join(data, how='outer', rsuffix='_dup')
                
                # Pause for 5 seconds to respect rate limits
                time.sleep(5)
            
            except Exception as e:
                flag += 1

                print(f"Error fetching data for {region} with keywords {chunk}: {e}")
                # Wait longer if an error occurs, then continue
                time.sleep(10)

            finally:
                if flag == 3:
                    break
    
    return all_data


In [12]:
# Convert keywords set to a list for chunking
keywords_list = list(keywords)

# Execute analysis
trend_data = analyze_search_trends(keywords_list, regions, timeframe)


# Display results
for region, data in trend_data.items():
    print(f"Trends for {region}:")
    print(data.head())
    print("\n")


c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future

Trends for IN:
                     aisle  headache  store  left  child  isPartial  \
date                                                                  
2025-03-14 12:00:00      0         7     83    23     27      False   
2025-03-14 13:00:00      0         7     81    21     26      False   
2025-03-14 14:00:00      0         6     76    23     29      False   
2025-03-14 15:00:00      0         6     70    22     26      False   
2025-03-14 16:00:00      0         6     58    22     25      False   

                     subscribe  whats  dragonball  high  others  \
date                                                              
2025-03-14 12:00:00          1     11           0    70       4   
2025-03-14 13:00:00          1     11           0    69       4   
2025-03-14 14:00:00          2     12           0    77       4   
2025-03-14 15:00:00          1     10           0    68       4   
2025-03-14 16:00:00          1      9           0    65       4   

                 

In [13]:
data

,aisle,headache,store,left,child,isPartial,subscribe,whats,dragonball,high,others,isPartial_dup,see,three,sixth,natural,steal,isPartial_dup
date,,,,,,,,,,,,,,,,,,
2025-03-14 12:00:00,0,7,83,23,27,False,1,11,0,70,4,False,82,64,4,48,2,False
2025-03-14 13:00:00,0,7,81,21,26,False,1,11,0,69,4,False,85,68,5,47,2,False
2025-03-14 14:00:00,0,6,76,23,29,False,2,12,0,77,4,False,83,69,5,49,2,False
2025-03-14 15:00:00,0,6,70,22,26,False,1,10,0,68,4,False,85,63,5,46,2,False
2025-03-14 16:00:00,0,6,58,22,25,False,1,9,0,65,4,False,80,60,4,39,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21 08:00:00,0,4,73,21,22,False,1,13,0,83,3,False,61,60,3,40,1,False
2025-03-21 09:00:00,0,4,75,21,22,False,1,13,0,88,3,False,59,52,3,41,2,False
2025-03-21 10:00:00,0,4,75,24,23,False,1,13,0,89,3,False,65,52,3,40,2,False


In [14]:
The scores in the trends data represent the relative search interest for the given keywords over time, as provided by Google Trends. Here's what they mean:

1. **Score Values**: Each score is a normalized value between 0 and 100, where:
    - `100` represents the peak popularity of the keyword during the specified timeframe.
    - `0` means there was insufficient data for that keyword at that time.

2. **Columns**: Each column corresponds to a keyword (e.g., `problem`, `poison`, etc.), and the values indicate the search interest for that keyword at the given timestamp.

3. **`isPartial` Columns**: These boolean columns indicate whether the data for that time period is incomplete or partial.

In summary, these scores help you understand how popular each keyword was in the specified region (`IN`) and timeframe (`now 7-d`).

SyntaxError: unterminated string literal (detected at line 1) (135476000.py, line 1)